In [1]:
include("../src/ProcessSequence.jl")

Main.ProcessSequence

In [2]:
na = "mouse_rna_single_cell/"

pafq = joinpath("../input/", na)

paou = joinpath("../output/", na)

pafa = "../input/reference_mouse_transcriptome/Mus_musculus.GRCm39.cdna.all.fa.gz"

n_jo = 2

println(pafq)

../input/mouse_rna_single_cell/


In [3]:
for fq1 in readdir(pafq; join = true)

    if occursin("_R1", fq1)
        
#         fq2 = replace(fq1, "R1" => "R2")

        na = split(string(split(fq1, ".fastq")[1]), "/")[4]
        
        paco = joinpath(paou, na)

        println(paco)
        
        ProcessSequence.count_transcript(
            fq1,
#             fq2,
            pafa,
            paco,
            n_jo::Int64,
        )

    end

end

../output/mouse_rna_single_cell/22702_WT-CD4-ACT-1_S1_L999_R1_001
(2021-07-26T19:48:04.767) Counting transcript ...
`kallisto quant --single -l 51 -s 0.05 --threads 2 --index ../input/reference_mouse_transcriptome/Mus_musculus.GRCm39.cdna.all.fa.gz.kallisto_index --output-dir ../output/mouse_rna_single_cell/22702_WT-CD4-ACT-1_S1_L999_R1_001 ../input/mouse_rna_single_cell/22702_WT-CD4-ACT-1_S1_L999_R1_001.fastq`



[quant] fragment length distribution is truncated gaussian with mean = 51, sd = 0.05
[index] k-mer length: 31
[index] number of targets: 0
[index] number of k-mers: 0
[index] number of equivalence classes: 0
[quant] running in single-end mode
[quant] will process file 1: ../input/mouse_rna_single_cell/22702_WT-CD4-ACT-1_S1_L999_R1_001.fastq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 55,326,961 reads, 0 reads pseudoaligned
[~warn] no reads pseudoaligned.
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 52 rounds
HDF5-DIAG: Error detected in HDF5 (1.10.6) thread 139796884449088:
  #000: H5Pdcpl.c line 2025 in H5Pset_chunk(): all chunk dimensions must be positive
    major: Invalid arguments to routine
    minor: Out of range
HDF5-DIAG: Error detected in HDF5 (1.10.6) thread 139796884449088:
  #000: H5D.c line 151 in H5Dcreate2(): unable to create dataset
    major: Dataset
    minor: Unable to initialize ob

LoadError: [91mfailed process: Process(`[4mkallisto[24m [4mquant[24m [4m--single[24m [4m-l[24m [4m51[24m [4m-s[24m [4m0.05[24m [4m--threads[24m [4m2[24m [4m--index[24m [4m../input/reference_mouse_transcriptome/Mus_musculus.GRCm39.cdna.all.fa.gz.kallisto_index[24m [4m--output-dir[24m [4m../output/mouse_rna_single_cell/22702_WT-CD4-ACT-1_S1_L999_R1_001[24m [4m../input/mouse_rna_single_cell/22702_WT-CD4-ACT-1_S1_L999_R1_001.fastq[24m`, ProcessExited(1)) [1][39m


## Make transcript by sample

In [ ]:
from environment import *

with open("setting.json") as io:

    SETTING = json.load(io)

PATH = make_path_dict(SETTING)

In [ ]:
enst_tpms = []

for sample_id in SETTING["fastq_id_fastq_file_paths"].keys():

    enst_tpm = pd.read_csv(
        os.path.join(PATH["kallisto/"], sample_id, "abundance.tsv"),
        sep="\t",
        index_col=0,
    )["tpm"]

    enst_tpm.name = sample_id

    enst_tpms.append(enst_tpm)

enst_x_sample = pd.concat(enst_tpms, axis=1)

enst_x_sample.index.name = "ENST"

enst_x_sample.to_csv(PATH["enst_x_sample.tsv"], sep="\t")

## Make gene by sample

In [ ]:
enst_gene_name = pd.read_csv(SETTING["enst_gene_name_file_path"], sep="\t")

gene_x_sample = pd.DataFrame(
    enst_x_sample,
    index=enst_x_sample.index.map(
        {
            enst: gene_name.str.upper()
            for enst, gene_name in zip(
                enst_gene_name["Transcript stable ID version"],
                enst_gene_name["Gene name"],
            )
        }
    ),
)

gene_x_sample.loc[~gene_x_sample.index.isna()].groupby(level=0).median()

gene_x_sample.index.name = "Gene"

gene_x_sample.to_csv(PATH["gene_x_sample.tsv"], sep="\t")